In [1]:
import networkx as nx
gB = nx.read_graphml('../Session_1/gB.graphml')
gD = nx.read_graphml('../Session_1/gD.graphml')

In [2]:
# calculate the number of weakly and strongly connected components in gB
num_weakly_connected_gB = nx.number_weakly_connected_components(gB)
num_strongly_connected_gB = nx.number_strongly_connected_components(gB)

# calculate the number of weakly and strongly connected components in gD
num_weakly_connected_gD = nx.number_weakly_connected_components(gD)
num_strongly_connected_gD = nx.number_strongly_connected_components(gD)

# print the results
print(f"gB - Number of weakly connected components: {num_weakly_connected_gB}")
print(f"gB - Number of strongly connected components: {num_strongly_connected_gB}")
print(f"gD - Number of weakly connected components: {num_weakly_connected_gD}")
print(f"gD - Number of strongly connected components: {num_strongly_connected_gD}")

gB - Number of weakly connected components: 1
gB - Number of strongly connected components: 372
gD - Number of weakly connected components: 1
gD - Number of strongly connected components: 393


In [4]:
gB_undirected = nx.read_graphml('gBp.graphml')
gD_undirected = nx.read_graphml('gDp.graphml')

In [5]:
# calculate the number of weakly and strongly connected components in gB
num_connected_gB_undirected = nx.number_connected_components(gB_undirected)

# calculate the number of weakly and strongly connected components in gD
num_connected_gD_undirected = nx.number_connected_components(gD_undirected)

# print the results
print(f"gB - Number of weakly connected components: {num_connected_gB_undirected}")
print(f"gD - Number of weakly connected components: {num_connected_gD_undirected}")


gB - Number of weakly connected components: 1
gD - Number of weakly connected components: 1
